In [ ]:
# telco_churn_shap_final


import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score
from sklearn.inspection import permutation_importance, PartialDependenceDisplay

# --------------- CONFIG ---------------
DATA_PATH = "C:\\Users\Hp\OneDrive\Desktop\WA_Fn-UseC_-Telco-Customer-Churn.csv"  # change if needed
OUTPUT_FOLDER = "D:\Cultus"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
RANDOM_STATE = 42

# --------------- helpers ---------------
def clean_numeric_string(val):
    """
    Clean numeric-like strings: remove brackets, parentheses, currency, commas and common NA tokens.
    """
    if pd.isna(val):
        return np.nan
    if not isinstance(val, str):
        return val
    s = val.strip()
    if s == "":
        return np.nan
    # Remove surrounding brackets or parentheses
    if (s.startswith("[") and s.endswith("]")) or (s.startswith("(") and s.endswith(")")):
        s = s[1:-1].strip()
    # Remove punctuation often present in numbers
    for ch in [",", "$", "₹", "%", '"', "'"]:
        s = s.replace(ch, "")
    if s.lower() in ("na", "n/a", "none", "nan", "null", "unknown", ""):
        return np.nan
    return s

def robust_convert_numeric(series):
    cleaned = series.astype(object).map(clean_numeric_string)
    coerced = pd.to_numeric(cleaned, errors='coerce')
    success_frac = coerced.notna().sum() / len(coerced)
    return coerced, success_frac

# --------------- 1. Load ---------------
print("Loading data:", DATA_PATH)
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)

# --------------- 2. Clean numeric columns ---------------
numeric_candidates = ['tenure', 'MonthlyCharges', 'TotalCharges']
for col in numeric_candidates:
    if col in df.columns:
        coerced, frac = robust_convert_numeric(df[col])
        print(f"Column {col} convertible fraction: {frac:.3f}")
        df[col] = coerced

# Auto-convert object columns that are mostly numeric (safety)
for col in df.select_dtypes(include=['object']).columns:
    if col.lower() in ('gender','partner','dependents','phoneservice','churn','customerid','contract','paymentmethod','internetservice'):
        continue
    coerced, frac = robust_convert_numeric(df[col])
    if frac > 0.85:
        print(f"Auto-converting {col} to numeric (frac={frac:.3f})")
        df[col] = coerced

# --------------- 3. Basic cleaning and imputation ---------------
if 'customerID' in df.columns:
    df = df.drop(columns=['customerID'])
if 'Churn' not in df.columns:
    raise ValueError("Missing Churn column")
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

# Impute numeric NaNs with median
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'Churn' in num_cols: num_cols.remove('Churn')
if len(num_cols) > 0:
    df[num_cols] = SimpleImputer(strategy='median').fit_transform(df[num_cols])

# Fill categorical missing
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
if len(cat_cols) > 0:
    df[cat_cols] = df[cat_cols].fillna("Missing")

# --------------- 4. Encoding ---------------
df_enc = pd.get_dummies(df, columns=cat_cols, drop_first=True)
X = df_enc.drop(columns=['Churn'])
y = df_enc['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=RANDOM_STATE)
print("Train/test shapes:", X_train.shape, X_test.shape)

# --------------- 5. Model training ---------------
use_xgb = False
try:
    import xgboost as xgb
    use_xgb = True
    print("XGBoost available")
except Exception as e:
    print("XGBoost not available:", e)

if use_xgb:
    # ensure param numeric and explicit base_score
    scale_pos_weight = float((y_train==0).sum() / (y_train==1).sum())
    model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='auc', random_state=RANDOM_STATE,
                              scale_pos_weight=scale_pos_weight, base_score=0.5, n_jobs=-1)
else:
    model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=RANDOM_STATE, n_jobs=-1)

model.fit(X_train, y_train)

# If XGBoost, set booster param to numeric string explicitly to avoid SHAP reading weird strings
if use_xgb:
    try:
        booster = model.get_booster()
        # set base_score to a numeric string that SHAP can parse
        booster.set_param({'base_score': '0.5'})
        print("Set booster.base_score to numeric string '0.5' to avoid SHAP parsing issues")
    except Exception as e:
        print("Could not set booster params:", e)

# --------------- 6. Evaluation ---------------
y_proba = model.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= 0.5).astype(int)
auc = roc_auc_score(y_test, y_proba)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
print(f"Metrics -> AUC: {auc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

# --------------- 7. Explainability: SHAP with robust fallbacks ---------------
shap_available = False
try:
    import shap
    shap_available = True
    print("SHAP version:", shap.__version__)
except Exception as e:
    print("SHAP not installed:", e)
    shap_available = False

shap_values_pos = None
global_imp = None

if shap_available:
    # Convert data to numeric numpy arrays to avoid object dtype issues in SHAP
    X_train_np = X_train.values.astype(np.float32)
    X_test_np = X_test.values.astype(np.float32)

    # Try TreeExplainer first (fast for trees). If it throws, fall back to shap.Explainer on predict_proba.
    try:
        explainer = shap.TreeExplainer(model)
        try:
            sv = explainer.shap_values(X_test_np)  # older API
        except Exception:
            sv = explainer(X_test_np)  # newer API
        if isinstance(sv, list):
            shap_values_pos = sv[1]
        else:
            try:
                shap_values_pos = np.array(sv.values)
            except Exception:
                shap_values_pos = np.array(sv)
        print("TreeExplainer succeeded")
    except Exception as e_tree:
        print("TreeExplainer failed:", type(e_tree).__name__, str(e_tree))
        print("Trying shap.Explainer with model.predict_proba and numpy arrays (slower)")
        try:
            predict_proba = lambda x: model.predict_proba(x)[:, 1]
            explainer2 = shap.Explainer(predict_proba, X_train_np)  # pass numpy
            sv2 = explainer2(X_test_np)
            # new shap.Explanation likely returns sv2.values with shape (n_samples, n_features)
            try:
                shap_values_pos = np.array(sv2.values)
            except Exception:
                shap_values_pos = np.array(sv2)
            print("shap.Explainer succeeded")
        except Exception as e_expl:
            print("shap.Explainer also failed:", type(e_expl).__name__, str(e_expl))
            shap_values_pos = None

    # If we have shap_values_pos, process and save plots
    if shap_values_pos is not None:
        shap_values_pos = np.asarray(shap_values_pos)
        # handle possible extra dims
        if shap_values_pos.ndim == 3:
            # try to reduce to (n_samples, n_features)
            shap_values_pos = shap_values_pos.reshape(shap_values_pos.shape[1], shap_values_pos.shape[2])

        mean_abs_shap = np.abs(shap_values_pos).mean(axis=0)
        global_imp = pd.Series(mean_abs_shap, index=X_test.columns).sort_values(ascending=False)
        print("Top 10 features by mean |SHAP|:")
        print(global_imp.head(10))

        # Save global mean|SHAP| bar
        plt.figure(figsize=(12,6))
        global_imp.head(20).plot.bar()
        plt.title("Global feature importance (mean |SHAP|) - top 20")
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_FOLDER, "global_shap_importance.png"))
        plt.close()

        # SHAP summary (beeswarm)
        try:
            plt.figure()
            shap.summary_plot(shap_values_pos, X_test_np, show=False)
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_FOLDER, "shap_summary_beeswarm.png"), bbox_inches='tight')
            plt.close()
        except Exception as e:
            print("Could not create SHAP beeswarm:", e)

        # Dependence plots for top-3 features
        top3 = global_imp.head(3).index.tolist()
        for feat in top3:
            try:
                plt.figure(figsize=(6,4))
                # shap.dependence_plot accepts DataFrame too; pass X_test (columns preserved) to label axes
                shap.dependence_plot(feat, shap_values_pos, X_test, show=False)
                plt.tight_layout()
                safe = feat.replace(' ', '_').replace('/', '_')
                plt.savefig(os.path.join(OUTPUT_FOLDER, f"shap_dependence_{safe}.png"))
                plt.close()
            except Exception as e:
                print(f"Dependence plot failed for {feat}:", e)

        # Local explanations: pick three representatives
        preds = (y_proba >= 0.5).astype(int)
        churn_idx = np.where((preds==1) & (y_test.values==1))[0]
        nonchurn_idx = np.where((preds==0) & (y_test.values==0))[0]
        borderline_idx = np.argmin(np.abs(y_proba - 0.5))
        selected = {
            "predicted_churn": int(X_test.index[churn_idx[0]]) if len(churn_idx)>0 else None,
            "predicted_nonchurn": int(X_test.index[nonchurn_idx[0]]) if len(nonchurn_idx)>0 else None,
            "borderline": int(X_test.index[borderline_idx])
        }
        print("Selected indices for local SHAP:", selected)

        for name, idx in selected.items():
            if idx is None:
                continue
            pos = list(X_test.index).index(idx)
            try:
                base_val = explainer.expected_value if 'explainer' in locals() and hasattr(explainer, "expected_value") else None
                ev = shap.Explanation(values=shap_values_pos[pos], base_values=base_val, data=X_test.iloc[pos])
                plt.figure(figsize=(10,4))
                shap.plots.waterfall(ev, show=False)
                plt.title(f"Local SHAP waterfall - {name} (idx {idx})")
                plt.tight_layout()
                plt.savefig(os.path.join(OUTPUT_FOLDER, f"shap_local_waterfall_{name}_{idx}.png"))
                plt.close()
            except Exception as e_local:
                # fallback: save top local SHAP values as CSV
                local_sh = pd.Series(shap_values_pos[pos], index=X_test.columns)
                local_sh.abs().sort_values(ascending=False).head(40).to_csv(
                    os.path.join(OUTPUT_FOLDER, f"shap_local_top_{name}_{idx}.csv")
                )
        print("SHAP artifacts saved to:", OUTPUT_FOLDER)
    else:
        print("SHAP computation did not produce values. Will use permutation importance fallback.")
        shap_available = False

# --------------- 8. Fallback (permutation importance + PDP) ---------------
if not shap_available or (shap_values_pos is None):
    print("Running permutation importance fallback.")
    r = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=RANDOM_STATE, n_jobs=1)
    perm_imp = pd.Series(r.importances_mean, index=X_test.columns).sort_values(ascending=False)
    perm_imp.head(20).to_csv(os.path.join(OUTPUT_FOLDER, "permutation_importance_top20.csv"))
    plt.figure(figsize=(12,6))
    perm_imp.head(20).plot.bar()
    plt.title("Permutation importance - top 20")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_FOLDER, "permutation_importance_top20.png"))
    plt.close()
    top3 = perm_imp.head(3).index.tolist()
    for feat in top3:
        try:
            fig, ax = plt.subplots(figsize=(6,4))
            PartialDependenceDisplay.from_estimator(model, X_test, [feat], ax=ax)
            plt.title(f"PDP - {feat}")
            plt.tight_layout()
            plt.savefig(os.path.join(OUTPUT_FOLDER, f"pdp_{feat.replace(' ', '_').replace('/', '_')}.png"))
            plt.close()
        except Exception as e:
            print("PDP failed for", feat, ":", e)
    # Local approximate contributions using feature importances
    fi = None
    try:
        fi = pd.Series(model.feature_importances_, index=X_train.columns)
    except Exception:
        pass
    preds = (y_proba >= 0.5).astype(int)
    churn_idx = np.where((preds==1) & (y_test.values==1))[0]
    nonchurn_idx = np.where((preds==0) & (y_test.values==0))[0]
    borderline_idx = np.argmin(np.abs(y_proba - 0.5))
    test_indices = X_test.index.to_numpy()
    selected = {
        "predicted_churn": int(test_indices[churn_idx[0]]) if len(churn_idx)>0 else None,
        "predicted_nonchurn": int(test_indices[nonchurn_idx[0]]) if len(nonchurn_idx)>0 else None,
        "borderline": int(test_indices[borderline_idx])
    }
    for name, idx in selected.items():
        if idx is None: continue
        pos = list(X_test.index).index(idx)
        row = X_test.iloc[pos]
        if fi is not None:
            contrib = (row - X_train.mean()) * fi
            contrib = contrib.sort_values(key=lambda x: np.abs(x), ascending=False).head(30)
            contrib.to_csv(os.path.join(OUTPUT_FOLDER, f"fallback_local_contrib_{name}_{idx}.csv"))
    print("Fallback artifacts saved to:", OUTPUT_FOLDER)

# --------------- 9. Summary file ---------------
with open(os.path.join(OUTPUT_FOLDER, "run_summary.txt"), "w") as f:
    f.write("Model evaluation:\n")
    f.write(f"AUC: {auc:.4f}\nPrecision: {precision:.4f}\nRecall: {recall:.4f}\nF1: {f1:.4f}\n\n")
    if shap_values_pos is not None:
        f.write("SHAP global and local outputs saved.\n")
    else:
        f.write("SHAP failed in this environment; permutation importance + PDP fallbacks saved.\n")
    f.write(f"Outputs folder: {OUTPUT_FOLDER}\n")

print("Done. Outputs in:", OUTPUT_FOLDER)


<>:23: SyntaxWarning: invalid escape sequence '\H'
<>:24: SyntaxWarning: invalid escape sequence '\C'
<>:23: SyntaxWarning: invalid escape sequence '\H'
<>:24: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Hp\AppData\Local\Temp\ipykernel_23420\2189942425.py:23: SyntaxWarning: invalid escape sequence '\H'
  DATA_PATH = "C:\\Users\Hp\OneDrive\Desktop\WA_Fn-UseC_-Telco-Customer-Churn.csv"  # change if needed
C:\Users\Hp\AppData\Local\Temp\ipykernel_23420\2189942425.py:24: SyntaxWarning: invalid escape sequence '\C'
  OUTPUT_FOLDER = "D:\Cultus"


Loading data: C:\Users\Hp\OneDrive\Desktop\WA_Fn-UseC_-Telco-Customer-Churn.csv
Shape: (7043, 21)
Column tenure convertible fraction: 1.000
Column MonthlyCharges convertible fraction: 1.000
Column TotalCharges convertible fraction: 0.998
Train/test shapes: (5282, 30) (1761, 30)
XGBoost available
Set booster.base_score to numeric string '0.5' to avoid SHAP parsing issues


d:\Cultus\Capstone_Project\venv\Lib\site-packages\xgboost\training.py:199: UserWarning: [11:26:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Metrics -> AUC: 0.8190, Precision: 0.5565, Recall: 0.6745, F1: 0.6099
SHAP version: 0.49.1
TreeExplainer failed: ValueError could not convert string to float: '[5E-1]'
Trying shap.Explainer with model.predict_proba and numpy arrays (slower)


PermutationExplainer explainer: 1762it [00:52, 31.63it/s]                          


...

<Figure size 600x400 with 0 Axes>

<Figure size 600x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>

<Figure size 1000x400 with 0 Axes>